# Invariant Testing

This document outlines useful snippets for writing agent trace assertions using the Invariant Testing library.

## Inspecting Traces

An agent run results in a trace of events and actions that correspond to the actions and responses of the agent.

For effective testing, we need to inspect the trace to ensure we are checking our test assertions against the correct parts of the trace.

The library provides a simple wrapper to inspect a given trace:


In [1]:
from invariant_runner.custom_types.trace import Trace

trace = Trace(trace=[
    {"role": "user", "content": "Hello there"},
    {"role": "assistant", "content": "Hello there", "tool_calls": [
        {
            "type": "function",
            "function": {
                "name": "greet",
                "arguments": {
                    "name": "there"
                }
            }
        }
    ]},
    {"role": "user", "content": "I need help with something."},
])

## Selecting Messages

A `Trace` object can be used to select specific messages from the trace. This is useful for selecting messages that are relevant to the test assertions.

In [5]:
# select the first trace message
trace.messages(0)

{'role': 'user', 'content': 'Hello there'} at 0

In [6]:
# select all user messages
trace.messages(role="user")

InvariantList[{'role': 'user', 'content': 'Hello there'}, {'role': 'user', 'content': 'I need help with something.'}] at [['0'], ['2']]

In [7]:
# select the message with 'something' in the content
trace.messages(content=lambda c: 'something' in c)

InvariantList[{'role': 'user', 'content': 'I need help with something.'}] at [['2']]

On the one hand, the `trace.messages(...)` selector function gives you a convenient way to select messages from the trace. However, in addition to this, it also keeps track of the exact path of the resulting objects in the trace.

This is useful for debugging and backtracking assertion failures, down to the exact agent event that is causing the failure.

Whenever accessing a message, property or sub-dictionary, Invariant Testing will track the exact path and range of its location in the trace.

In [14]:
# selecting content from the 2nd message in the trace
content = trace.messages(2)["content"]
print(content)

I need help with something. at 2.content:0-27


## Selecting Tool Calls

Similar to selecting messages, you can also select just tool calls from the trace.

In [3]:
greet_calls = trace.tool_calls(name="greet")
print(greet_calls[0])

{'type': 'function', 'function': {'name': 'greet', 'arguments': {'name': 'there'}}} at ['1.tool_calls.0']


Again, all accesses to tool calls are tracked and include the exact source path and range in the trace (`1.tool_calls.0` here).

Note that even though you can select `.tool_calls()` directly on `name` and `arguments`, the returned object is always of `{'type': 'function', 'function': { ... }}` shape.